In [57]:
import seaborn as sns
import folium
import folium.plugins as folium_plugins
from geopandas import GeoSeries, GeoDataFrame
import geopandas as gpd
import pandas as pd
from os.path import exists


from utils.downloads import (
    Censo,
    Nivel,
    download_malha
)
from utils.geosampa_client import get_client
from utils.geo import calc_similarity

In [3]:
sns.set_theme(rc={'figure.figsize':(11.7,8.27)})

# Carregando os dados necessários

Para facilitar o desenvolvimento posterior, vamos concentrar os downloads e carregamentos iniciais nesta seção.

## Setores censitários de 2022

In [4]:
%%time
setores22 = download_malha(Censo.CENSO_2022, Nivel.SETORES)
setores22 = setores22[setores22['CD_MUN'] == '3550308']
setores22 = setores22.to_crs(epsg=31983)
setores22.sample(3)

INFO:root:Carregando a malha de setores do censo de 2022.
INFO:root:O arquivo data/cache/setores/2022/SP_Malha_Preliminar_2022.zip já foi baixado anteriormente. Usando cache local.


CPU times: user 1min 3s, sys: 746 ms, total: 1min 4s
Wall time: 1min 6s


,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,v0001,v0002,v0003,v0004,v0005,v0006,v0007,geometry
92676,355030894000189P,0.008016,3,Sudeste,35,São Paulo,3550308,São Paulo,355030894,Vila Sônia,...,3550308,São Paulo/SP,198,130,130,0,2.000000,21.212121,99,"POLYGON ((322645.353 7387603.330, 322647.045 7..."
66144,355030803000156P,0.038809,3,Sudeste,35,São Paulo,3550308,São Paulo,355030803,Anhanguera,...,3550308,São Paulo/SP,511,184,184,0,3.059880,8.383234,167,"POLYGON ((316660.632 7408174.552, 316658.985 7..."
70458,355030821000058P,0.089724,3,Sudeste,35,São Paulo,3550308,São Paulo,355030821,Casa Verde,...,3550308,São Paulo/SP,381,199,199,0,2.540000,25.333333,150,"POLYGON ((331515.929 7398794.371, 331479.729 7..."


## Setores censitários de 2010

In [5]:
setores10 = download_malha(Censo.CENSO_2010, Nivel.SETORES)
setores10 = setores10[setores10['CD_GEOCODM'] == '3550308']
setores10 = setores10.to_crs(epsg=31983)
setores10.sample(3)

INFO:root:Carregando a malha de setores do censo de 2010.
INFO:root:O arquivo data/cache/setores/2010/sp_setores_censitarios.zip já foi baixado anteriormente. Usando cache local.


,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
48582,114075.0,355030829000162,URBANO,35503082900,None,355030829,FREGUESIA DO Ó,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,48583,"POLYGON ((325939.703 7404330.475, 325946.268 7..."
55575,121129.0,355030862000066,URBANO,35503086200,None,355030862,PINHEIROS,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,55576,"POLYGON ((327899.633 7392507.856, 327918.696 7..."
50719,116197.0,355030837000068,URBANO,35503083700,None,355030837,ITAQUERA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,50720,"POLYGON ((351967.009 7397828.877, 351953.309 7..."


## Quadras viárias

In [6]:
quadra_fiscal_path = 'data/quadra_viaria_editada.parquet'
if exists(quadra_fiscal_path):
    gdf_quadras = gpd.read_parquet(quadra_fiscal_path)
else:
    raw_features = gs_client.get_feature('quadra_viaria_editada')
    gdf_quadras = GeoDataFrame.from_features(raw_features['features'])
    gdf_quadras = gdf_quadras.set_crs(raw_features['crs'].get('properties').get('name'))
    gdf_quadras = gdf_quadras.to_crs(epsg=31983)
    gdf_quadras.to_parquet(quadra_fiscal_path)
gdf_quadras

,geometry,cd_identificador_quadra_viaria_editada,cd_identificador_quadra_viaria_editada_original,tx_tipo_quadra_viaria,tx_escala,tx_ano_referencia,qt_area_metro,sg_fonte_original
0,"POLYGON ((338344.482 7407584.661, 338344.543 7...",1,52943,Quadra,1:1.000,2004,7650,SMUL/GEOINFO
1,"POLYGON ((327186.316 7377154.793, 327188.886 7...",2,4,Praca_Canteiro,1:1.000,2004,11,SMUL/GEOINFO
2,"POLYGON ((324545.386 7402131.104, 324544.170 7...",3,5,Praca_Canteiro,1:1.000,2004,6,SMUL/GEOINFO
3,"POLYGON ((326273.589 7399282.971, 326273.013 7...",4,6,Praca_Canteiro,1:1.000,2004,109,SMUL/GEOINFO
4,"POLYGON ((322149.447 7392535.031, 322154.771 7...",5,7,Praca_Canteiro,1:1.000,2004,12,SMUL/GEOINFO
...,...,...,...,...,...,...,...,...
64779,"POLYGON ((325071.550 7397335.276, 325073.371 7...",64780,0,Praca_Canteiro,1:1.000,2017,0,SMUL/GEOINFO
64780,"POLYGON ((325064.631 7397341.102, 325065.814 7...",64781,0,Praca_Canteiro,1:1.000,2017,0,SMUL/GEOINFO
64781,"POLYGON ((325102.327 7397333.002, 325089.121 7...",64782,58914,Praca_Canteiro,1:1.000,2004,25982,SMUL/GEOINFO
64782,"POLYGON ((325081.631 7397226.147, 325087.148 7...",64783,0,Praca_Canteiro,1:1.000,2017,0,SMUL/GEOINFO


In [10]:
gdf_quadras.head()

,geometry,cd_identificador_quadra_viaria_editada,cd_identificador_quadra_viaria_editada_original,tx_tipo_quadra_viaria,tx_escala,tx_ano_referencia,qt_area_metro,sg_fonte_original
0,"POLYGON ((338344.482 7407584.661, 338344.543 7...",1,52943,Quadra,1:1.000,2004,7650,SMUL/GEOINFO
1,"POLYGON ((327186.316 7377154.793, 327188.886 7...",2,4,Praca_Canteiro,1:1.000,2004,11,SMUL/GEOINFO
2,"POLYGON ((324545.386 7402131.104, 324544.170 7...",3,5,Praca_Canteiro,1:1.000,2004,6,SMUL/GEOINFO
3,"POLYGON ((326273.589 7399282.971, 326273.013 7...",4,6,Praca_Canteiro,1:1.000,2004,109,SMUL/GEOINFO
4,"POLYGON ((322149.447 7392535.031, 322154.771 7...",5,7,Praca_Canteiro,1:1.000,2004,12,SMUL/GEOINFO


Para garantir, vamos conferir o crs e garantir todos os datasets utilizam o mesmo sistema.

In [7]:
setores10.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
setores22.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
gdf_quadras.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

# Preparando os dados

## Aplicando buffers negativos nos setores censitários

In [18]:
setores22_debuff = setores22.copy()
setores22_debuff['geometry'] = setores22_debuff.buffer(-2)

## Removendo quadras não habitadas

In [22]:
gdf_quadras['tx_tipo_quadra_viaria'].unique()

array(['Quadra', 'Praca_Canteiro', 'CET', 'Borda', 'Ilha'], dtype=object)

In [23]:
quadras_viarias = gdf_quadras.query('tx_tipo_quadra_viaria == "Quadra"').copy()
quadras_viarias.head()

,geometry,cd_identificador_quadra_viaria_editada,cd_identificador_quadra_viaria_editada_original,tx_tipo_quadra_viaria,tx_escala,tx_ano_referencia,qt_area_metro,sg_fonte_original
0,"POLYGON ((338344.482 7407584.661, 338344.543 7...",1,52943,Quadra,1:1.000,2004,7650,SMUL/GEOINFO
20,"POLYGON ((320433.366 7395346.357, 320474.980 7...",21,32,Quadra,1:1.000,2004,5372,SMUL/GEOINFO
23,"POLYGON ((320527.185 7395368.353, 320526.962 7...",24,35,Quadra,1:1.000,2004,5476,SMUL/GEOINFO
25,"POLYGON ((320533.150 7395370.806, 320576.085 7...",26,37,Quadra,1:1.000,2004,5941,SMUL/GEOINFO
27,"POLYGON ((320631.706 7395394.505, 320636.887 7...",28,39,Quadra,1:1.000,2004,7033,SMUL/GEOINFO


# Avaliando as similaridades com base nas quadras viárias

## Visualizando uma amostra aleatória

In [24]:
calc_similarity(quadras_viarias.sample(1), setores22_debuff, left_key_col='cd_identificador_quadra_viaria_editada', right_key_col='CD_SETOR')

,cd_identificador_quadra_viaria_editada,CD_SETOR,geometry,inter_area,inter_perc
79717,36893,355030846000260P,"POLYGON ((323016.353 7383197.519, 323016.440 7...",7701.725469,0.343072
79724,36893,355030846000270P,"POLYGON ((323021.019 7383198.859, 323046.802 7...",14747.554889,0.656928


In [25]:
calc_similarity(setores22_debuff.sample(1), quadras_viarias, left_key_col='CD_SETOR', right_key_col='cd_identificador_quadra_viaria_editada')

,CD_SETOR,cd_identificador_quadra_viaria_editada,geometry,inter_area,inter_perc
50500,355030840000064P,50501,"POLYGON ((320257.737 7399353.334, 320262.473 7...",15559.177046,0.501642
50589,355030840000064P,50590,"POLYGON ((320127.801 7399396.842, 320115.141 7...",8208.609853,0.264653
51696,355030840000064P,51697,"POLYGON ((320164.983 7399525.329, 320167.311 7...",7248.701840,0.233705


In [27]:
m = setores22[setores22['CD_SETOR'].isin(['355030840000064P'])].explore(
    name='Setores (2022)',  # name of the layer in the map
    color='royalblue',
    style_kwds={'fill': False},
)
m = gdf_quadras[gdf_quadras['cd_identificador_quadra_viaria_editada'].isin([50501, 50590, 51697])].explore(
    m=m,
    name='Quadras viárias',  # name of the layer in the map
    color='purple',
)

folium.LayerControl().add_to(m)  # use folium to add layer control

m

## Calculando a composição das quadras

In [30]:
%%time
quadras_composition_path = 'data/11_quadras_composition.parquet'
if exists(quadras_composition_path):
    comp_df = gpd.read_parquet(quadras_composition_path)
else:
    comp_df = calc_similarity(quadras_viarias, setores22_debuff, left_key_col='cd_identificador_quadra_viaria_editada', right_key_col='CD_SETOR')

CPU times: user 555 ms, sys: 108 ms, total: 663 ms
Wall time: 687 ms


In [31]:
comp_df

,cd_identificador_quadra_viaria_editada,CD_SETOR,geometry,inter_area,inter_perc
89339,1,355030881000338P,"POLYGON ((338344.543 7407586.495, 338344.890 7...",7649.167652,1.000000
77548,21,355030841000092P,"POLYGON ((320474.980 7395356.768, 320476.196 7...",5372.137667,1.000000
77548,24,355030841000092P,"POLYGON ((320526.962 7395367.967, 320526.872 7...",5476.762509,1.000000
77548,26,355030841000092P,"POLYGON ((320576.085 7395380.678, 320578.024 7...",5941.258822,1.000000
77548,28,355030841000092P,"POLYGON ((320636.887 7395372.752, 320652.850 7...",7033.083482,1.000000
...,...,...,...,...,...
75135,64784,355030833000353P,"MULTIPOLYGON (((355011.251 7389932.753, 355011...",62095.383621,0.058578
75136,64784,355030833000354P,"MULTIPOLYGON (((355261.512 7389451.321, 355254...",18509.597432,0.017461
75137,64784,355030833000355P,"MULTIPOLYGON (((355234.321 7389581.381, 355235...",14208.916653,0.013404
75149,64784,355030833000367P,"POLYGON ((355941.048 7387947.593, 355941.109 7...",18267.972041,0.017233


Primeiro, vamos garantir que todas as interseções sejam compostas apenas por polígonos

In [32]:
comp_df['geometry'].type.value_counts()

Polygon         68191
MultiPolygon     6205
Name: count, dtype: int64

Como não existem GeometryCollections, seguimos com as análises.

In [33]:
comp_df.to_parquet(quadras_composition_path)

## Calculando a similaridade das quadras

In [34]:
%%time
quadras_similarity_path = 'data/11_quadras_similarity.parquet'
if exists(quadras_similarity_path):
    sim_df = gpd.read_parquet(quadras_similarity_path)
else:
    sim_df = calc_similarity(setores22_debuff, quadras_viarias, left_key_col='CD_SETOR', right_key_col='cd_identificador_quadra_viaria_editada')

CPU times: user 47min 3s, sys: 996 ms, total: 47min 4s
Wall time: 47min 4s


In [35]:
sim_df

,CD_SETOR,cd_identificador_quadra_viaria_editada,geometry,inter_area,inter_perc
38479,355030801000001P,38480,"POLYGON ((339624.362 7392759.970, 339655.406 7...",2423.419273,0.041683
38548,355030801000001P,38549,"POLYGON ((339527.710 7392818.881, 339528.951 7...",11017.294210,0.189500
39168,355030801000001P,39169,"POLYGON ((339719.044 7392694.660, 339743.021 7...",11605.305329,0.199614
39181,355030801000001P,39182,"POLYGON ((339605.452 7392737.598, 339605.898 7...",14457.293542,0.248668
55865,355030801000001P,55866,"POLYGON ((339689.654 7392784.177, 339664.214 7...",18635.535290,0.320535
...,...,...,...,...,...
20898,355030896000338P,20899,"POLYGON ((355571.342 7397652.973, 355572.047 7...",3397.302028,0.129597
21765,355030896000338P,21766,"POLYGON ((355569.586 7397689.532, 355569.364 7...",10270.063319,0.391773
20537,355030896000339P,20538,"POLYGON ((355804.621 7397538.952, 355799.039 7...",957.917148,0.129406
20898,355030896000339P,20899,"POLYGON ((355570.911 7397640.723, 355571.163 7...",982.700260,0.132754


Primeiro, vamos garantir que todas as interseções sejam compostas apenas por polígonos

In [36]:
sim_df['geometry'].type.value_counts()

Polygon         68191
MultiPolygon     6205
Name: count, dtype: int64

Como não existem GeometryCollections, seguimos com as análises.

In [37]:
sim_df.to_parquet(quadras_similarity_path)

# Padronizando os nomes das variáveis

Para comparação, as variáveis mais interessantes são Total de pessoas e Total de domicílios.

No censo de 2022 essas variáveis são, respectivamente, V0001 e V0002.

No censo de 2010 essas variáveis são, respectivamente, a V001 do arquivo Domicilio02_SP1.xls e a V001 do arquivo Domicilio01_SP1.XLS.

Vamos trazer as variáveis dos dataframes auxiliares do censo de 2010 e padronizar os nomes.

In [40]:
setores22 = setores22.rename(columns={'v0001':'pop_total', 'v0002': 'dom_total'})
setores22.head()

,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,pop_total,dom_total,v0003,v0004,v0005,v0006,v0007,geometry
65763,355030801000001P,0.071797,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,682,329,329,0,2.359862,3.806228,289,"POLYGON ((339819.409 7392592.743, 339817.380 7..."
65764,355030801000002P,0.071902,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,1374,1011,1011,0,2.293823,1.502504,599,"POLYGON ((339965.618 7393034.241, 339934.552 7..."
65765,355030801000003P,0.055681,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,557,239,238,1,2.673171,9.268293,205,"POLYGON ((340157.893 7392942.288, 340105.133 7..."
65766,355030801000004P,0.064905,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,526,245,245,0,2.380090,1.357466,221,"POLYGON ((339898.252 7392661.977, 339885.527 7..."
65767,355030801000005P,0.086820,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,579,258,258,0,2.619910,9.049774,221,"POLYGON ((339650.828 7392494.453, 339535.589 7..."


Por último, vamos converter os tipos das colunas das variáveis alvo.

In [41]:
setores22['pop_total'] = setores22['pop_total'].astype(int)
setores22['dom_total'] = setores22['dom_total'].astype(int)
setores22.head()

,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,pop_total,dom_total,v0003,v0004,v0005,v0006,v0007,geometry
65763,355030801000001P,0.071797,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,682,329,329,0,2.359862,3.806228,289,"POLYGON ((339819.409 7392592.743, 339817.380 7..."
65764,355030801000002P,0.071902,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,1374,1011,1011,0,2.293823,1.502504,599,"POLYGON ((339965.618 7393034.241, 339934.552 7..."
65765,355030801000003P,0.055681,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,557,239,238,1,2.673171,9.268293,205,"POLYGON ((340157.893 7392942.288, 340105.133 7..."
65766,355030801000004P,0.064905,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,526,245,245,0,2.380090,1.357466,221,"POLYGON ((339898.252 7392661.977, 339885.527 7..."
65767,355030801000005P,0.086820,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,3550308,São Paulo/SP,579,258,258,0,2.619910,9.049774,221,"POLYGON ((339650.828 7392494.453, 339535.589 7..."


# Estimativas

## Calculando as variáveis correspondentes a cada interseção

In [42]:
est_2022 = sim_df.merge(setores22[['CD_SETOR', 'pop_total', 'dom_total']])
est_2022.head()

,CD_SETOR,cd_identificador_quadra_viaria_editada,geometry,inter_area,inter_perc,pop_total,dom_total
0,355030801000001P,38480,"POLYGON ((339624.362 7392759.970, 339655.406 7...",2423.419273,0.041683,682,329
1,355030801000001P,38549,"POLYGON ((339527.710 7392818.881, 339528.951 7...",11017.294210,0.189500,682,329
2,355030801000001P,39169,"POLYGON ((339719.044 7392694.660, 339743.021 7...",11605.305329,0.199614,682,329
3,355030801000001P,39182,"POLYGON ((339605.452 7392737.598, 339605.898 7...",14457.293542,0.248668,682,329
4,355030801000001P,55866,"POLYGON ((339689.654 7392784.177, 339664.214 7...",18635.535290,0.320535,682,329


In [43]:
est_2022['pop_total'] = round(est_2022['pop_total']*est_2022['inter_perc'], 0).astype(int)
est_2022['dom_total'] = round(est_2022['dom_total']*est_2022['inter_perc'], 0).astype(int)
est_2022.head()

,CD_SETOR,cd_identificador_quadra_viaria_editada,geometry,inter_area,inter_perc,pop_total,dom_total
0,355030801000001P,38480,"POLYGON ((339624.362 7392759.970, 339655.406 7...",2423.419273,0.041683,28,14
1,355030801000001P,38549,"POLYGON ((339527.710 7392818.881, 339528.951 7...",11017.294210,0.189500,129,62
2,355030801000001P,39169,"POLYGON ((339719.044 7392694.660, 339743.021 7...",11605.305329,0.199614,136,66
3,355030801000001P,39182,"POLYGON ((339605.452 7392737.598, 339605.898 7...",14457.293542,0.248668,170,82
4,355030801000001P,55866,"POLYGON ((339689.654 7392784.177, 339664.214 7...",18635.535290,0.320535,219,105


## Estimando a população por quadra viária

In [45]:
est_2022_agg = est_2022[['cd_identificador_quadra_viaria_editada', 'pop_total', 'dom_total']].groupby('cd_identificador_quadra_viaria_editada').sum().reset_index()
est_2022_agg.head()

,cd_identificador_quadra_viaria_editada,pop_total,dom_total
0,1,154,55
1,21,42,17
2,24,43,17
3,26,46,18
4,28,55,22


In [46]:
pop_quadras_2022 = quadras_viarias.merge(est_2022_agg, how='left')
pop_quadras_2022.head()

,geometry,cd_identificador_quadra_viaria_editada,cd_identificador_quadra_viaria_editada_original,tx_tipo_quadra_viaria,tx_escala,tx_ano_referencia,qt_area_metro,sg_fonte_original,pop_total,dom_total
0,"POLYGON ((338344.482 7407584.661, 338344.543 7...",1,52943,Quadra,1:1.000,2004,7650,SMUL/GEOINFO,154.0,55.0
1,"POLYGON ((320433.366 7395346.357, 320474.980 7...",21,32,Quadra,1:1.000,2004,5372,SMUL/GEOINFO,42.0,17.0
2,"POLYGON ((320527.185 7395368.353, 320526.962 7...",24,35,Quadra,1:1.000,2004,5476,SMUL/GEOINFO,43.0,17.0
3,"POLYGON ((320533.150 7395370.806, 320576.085 7...",26,37,Quadra,1:1.000,2004,5941,SMUL/GEOINFO,46.0,18.0
4,"POLYGON ((320631.706 7395394.505, 320636.887 7...",28,39,Quadra,1:1.000,2004,7033,SMUL/GEOINFO,55.0,22.0


## Calculando a densidade demográfica por quadra viária

In [52]:
pop_quadras_2022['dens_22'] = pop_quadras_2022['pop_total']/(pop_quadras_2022['qt_area_metro']/1e6)
pop_quadras_2022

,geometry,cd_identificador_quadra_viaria_editada,cd_identificador_quadra_viaria_editada_original,tx_tipo_quadra_viaria,tx_escala,tx_ano_referencia,qt_area_metro,sg_fonte_original,pop_total,dom_total,dens_22
0,"POLYGON ((338344.482 7407584.661, 338344.543 7...",1,52943,Quadra,1:1.000,2004,7650,SMUL/GEOINFO,154.0,55.0,20130.718954
1,"POLYGON ((320433.366 7395346.357, 320474.980 7...",21,32,Quadra,1:1.000,2004,5372,SMUL/GEOINFO,42.0,17.0,7818.317200
2,"POLYGON ((320527.185 7395368.353, 320526.962 7...",24,35,Quadra,1:1.000,2004,5476,SMUL/GEOINFO,43.0,17.0,7852.447042
3,"POLYGON ((320533.150 7395370.806, 320576.085 7...",26,37,Quadra,1:1.000,2004,5941,SMUL/GEOINFO,46.0,18.0,7742.804242
4,"POLYGON ((320631.706 7395394.505, 320636.887 7...",28,39,Quadra,1:1.000,2004,7033,SMUL/GEOINFO,55.0,22.0,7820.275842
...,...,...,...,...,...,...,...,...,...,...,...
46622,"POLYGON ((322698.321 7376505.161, 322693.377 7...",64768,95434,Quadra,1:1.000,2017,5596246,SMUL/GEOINFO,8159.0,3398.0,1457.941627
46623,"POLYGON ((352981.150 7384916.491, 352969.995 7...",64771,77025,Quadra,1:1.000,2017,57522,SMUL/GEOINFO,875.0,324.0,15211.571225
46624,"POLYGON ((352948.915 7384912.955, 352950.852 7...",64772,77025,Quadra,1:1.000,2017,57522,SMUL/GEOINFO,111.0,42.0,1929.696464
46625,"POLYGON ((353495.362 7390786.169, 353483.339 7...",64773,96124,Quadra,1:1.000,2017,724837,SMUL/GEOINFO,1525.0,614.0,2103.921295


# Visualizando a estimativa de população

In [53]:
def create_map(df:gpd.GeoDataFrame, col_mapper:dict, legend_column:str, base:str='2022', **kwargs) -> folium.Map:
    
    df = df[[k for k in col_mapper]].copy()
    # df = df[df['pop_total'].astype(int) > 0]
    df = df.rename(columns=col_mapper)
    if base == '2022':
        # m = setores10[['CD_GEOCODI', 'pop_total', 'geometry']].explore(
        #     tiles="CartoDB positron",  # use "CartoDB positron" tiles
        #     name='Setores censitários (2010)',
        #     color='grey',
        #     style_kwds={'fillOpacity': 0.1},
        # )
        
        # m = est_2010[['CD_SETOR', 'CD_GEOCODI', 'pop_total', 'geometry']].explore(
        #     m=m,
        #     name='Similaridades',
        #     color='grey',
        #     style_kwds={'fillOpacity': 0.1},
        #     show=False,
        # )
        
        m = setores22[['CD_SETOR', 'pop_total', 'geometry']].explore(
            tiles="CartoDB positron",  # use "CartoDB positron" tiles
            name='Setores censitários (2022)',
            color='grey',
            style_kwds={'fillOpacity': 0.1},
        )
    
        m = setores22[['geometry']].dissolve().explore(
            m=m,
            name='Limite municipal',
            color='black',
            style_kwds={'fill': False},
        )
    elif base == '2010':
        m = setores22[['CD_SETOR', 'pop_total', 'geometry']].explore(
            tiles="CartoDB positron",  # use "CartoDB positron" tiles
            name='Setores censitários (2022)',
            color='grey',
            style_kwds={'fillOpacity': 0.1},
        )
        
        m = est_2022[['CD_GEOCODI', 'CD_SETOR', 'pop_total', 'geometry']].explore(
            m=m,
            name='Similaridades',
            color='grey',
            style_kwds={'fillOpacity': 0.1},
            show=False,
        )
    
        m = setores10[['geometry']].dissolve().explore(
            m=m,
            name='Limite municipal',
            color='black',
            style_kwds={'fill': False},
        )

    defaultKwargs = {
        # 'scheme': 'quantiles',
        'cmap': 'coolwarm',
        # 'k': 10
    }
    kwargs = { **defaultKwargs, **kwargs }

    popup_cols = [
        # 'Setor censitário',
        # 'Distrito',
        # 'População total (2010)',
        'População total (2022)',
        # 'Variação populacional (absoluta)',
        # 'Variação populacional (%)',
        'Densidade demográfica (hab/km^2)',
        # 'Variação na densidade demográfica (hab/km^2)',
        # 'Variação na densidade demográfica normalizada (raiz cúbica)',
    ]
        
    m = df.explore(
        m=m,
        column=legend_column,  # make choropleth based on "POP2010" column
        legend=True,  # show legend
        tooltip=popup_cols,
        popup=popup_cols,
        # legend_kwds=dict(colorbar=True, scale=False),  # do not use colorbar
        legend_kwds=dict(colorbar=True),  # do not use colorbar
        name=legend_column,  # name of the layer in the map,
        **kwargs
    )
    
    folium.LayerControl().add_to(m)

    folium_plugins.Fullscreen(
        position='topright',
        title='Tela cheia',
        title_cancel='Exit me',
        force_separate_button=True,
    ).add_to(m)

    return m

In [54]:
mapper_22 = {
    'cd_identificador_quadra_viaria_editada': 'Quadra viária',
    'pop_total': 'População total (2022)',
    # 'pop_total_prev': 'População total (2010)',
    # 'var_pop': 'Variação populacional (absoluta)',
    # 'var_pop_perc': 'Variação populacional (percentual)',
    # 'var_pop_perc_str': 'Variação populacional (%)',
    # 'var_dens': 'Variação na densidade demográfica (hab/km^2)',
    # 'cbrt_var_dens': 'Variação na densidade demográfica normalizada (raiz cúbica)',
    'dens_22': 'Densidade demográfica (hab/km^2)',
    'geometry': 'geometry'
}

In [58]:
%%time
m = create_map(pop_quadras_2022, mapper_22, 'Densidade demográfica (hab/km^2)')

CPU times: user 19 s, sys: 641 ms, total: 19.6 s
Wall time: 19.9 s


In [60]:
%%time
m.save('plots/Densidade demográfica - Quadras viárias.html')

CPU times: user 27.6 s, sys: 2.11 s, total: 29.7 s
Wall time: 34.5 s


In [61]:
%%time
m = create_map(pop_quadras_2022, mapper_22, 'Densidade demográfica (hab/km^2)', scheme='userdefined', classification_kwds={'bins': [1e3, 5e3, 1e4, 1e5, 1e6, 5e6]})

CPU times: user 25.2 s, sys: 3.02 s, total: 28.2 s
Wall time: 2min 38s


In [63]:
%%time
m.save('plots/Densidade demográfica - Quadras viárias - categorias.html')

CPU times: user 28 s, sys: 2.59 s, total: 30.6 s
Wall time: 35.8 s
